In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import os
import random

from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from models.vaes import Base, VAE, IWAE, AMCVAE, LMCVAE, VAE_with_flows
from models.samplers import HMC, MALA, ULA, run_chain
from utils import make_dataloaders
import yaml
import numpy as np

import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers

%matplotlib widget

In [2]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
pl.__version__

'1.1.3'

In [4]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [5]:
def generate_dataset(N, eps0=None, d=2, sigma=1.):
    seed_everything(42)
    z = np.random.randn(N, d)
    x = 2 * np.pi * (np.linalg.norm(z, axis=1, keepdims=True) + 2.) + np.random.randn(N, 1) * sigma
    return x

In [6]:
class Toy(Base):   
    def joint_logdensity(self, use_true_decoder=None):
        def density(z, x):
            if (use_true_decoder is not None) and use_true_decoder:
                x_reconst = self(z)
            elif hasattr(self, 'use_cloned_decoder') and self.use_cloned_decoder:
                x_reconst = self.cloned_decoder(z)
            else:
                x_reconst = self(z)
            log_Pr = torch.distributions.Normal(loc=torch.tensor(0., device=x.device, dtype=torch.float32),
                                    scale=torch.tensor(1., device=x.device, dtype=torch.float32)).log_prob(
                    z).sum(-1)
            
            return torch.distributions.Normal(loc=2 * np.pi * (torch.sqrt(torch.sum(torch.pow(z, 2), dim=1, keepdim=True)) + 2) + 0. * self.decoder_net.aux,
                                                  scale=self.decoder_net.sigma).log_prob(x).sum(-1) + log_Pr

        return density
    

class VAE_Toy(VAE, Toy):
    def loss_function(self, recon_x, x, mu, logvar):
        batch_size = mu.shape[0] // self.num_samples
        MSE = F.mse_loss(recon_x.view(mu.shape[0], -1), x.view(mu.shape[0], -1),
                                                 reduction='none').view(
            (self.num_samples, batch_size, -1)).mean(0).sum(-1).mean()
        KLD = -0.5 * torch.mean((1 + logvar - mu.pow(2) - logvar.exp()).view(
            (self.num_samples, -1, self.hidden_dim)).mean(0).sum(-1))
        loss = MSE + KLD

        return loss
    
class IWAE_Toy(IWAE, Toy):
    def loss_function(self, recon_x, x, mu, logvar, z):
        batch_size = mu.shape[0] // self.num_samples
        self.hidden_dim = mu.shape[1]
        log_Q = torch.distributions.Normal(loc=mu,
                                           scale=torch.exp(0.5 * logvar)).log_prob(z).view(
            (self.num_samples, -1, self.hidden_dim)).sum(-1)

        log_Pr = torch.sum((-0.5 * torch.abs(z).pow(2.)).view((self.num_samples, -1, self.hidden_dim)), -1)
        MSE = F.mse_loss(recon_x.view(mu.shape[0], -1), x.view(mu.shape[0], -1),
                                                 reduction='none').view(
            (self.num_samples, batch_size, -1)).sum(-1)

        log_weight = log_Pr - MSE - log_Q
        log_weight = log_weight - torch.max(log_weight, 0)[0]  # for stability
        weight = torch.exp(log_weight)
        weight = weight / torch.sum(weight, 0)
        weight = weight.detach()
        loss = torch.mean(torch.sum(weight * (-log_Pr + MSE + log_Q), 0))

        return loss
    
class VAE_with_flows_Toy(VAE_with_flows, Toy):
    def loss_function(self, recon_x, x, mu, logvar, z, z_transformed, log_jac):
        batch_size = mu.shape[0] // self.num_samples
        MSE = F.mse_loss(recon_x.view(mu.shape[0], -1), x.view(mu.shape[0], -1),
                                                 reduction='none').view(
            (self.num_samples, batch_size, -1)).mean(0).sum(-1).mean()
        log_Q = torch.mean(torch.distributions.Normal(loc=mu, scale=torch.exp(0.5 * logvar)).log_prob(z).view(
            (self.num_samples, batch_size, -1)).sum(-1) - log_jac.view((self.num_samples, -1)), dim=0).mean()
        log_Pr = (-0.5 * z_transformed ** 2).view(
            (self.num_samples, batch_size, -1)).mean(0).sum(-1).mean()
        KLD = log_Q - log_Pr
        loss = MSE + KLD

        return loss
    
class LMCVAE_Toy(LMCVAE, Toy):
    def loss_function(self, sum_log_weights):
        loss = super(LMCVAE_Toy, self).loss_function(sum_log_weights)

        return loss

class AMCVAE_Toy(AMCVAE, Toy):
    def loss_function(self, sum_log_alphas, sum_log_weights):
        loss = super(AMCVAE_Toy, self).loss_function(sum_log_alphas, sum_log_weights)

        return loss

In [7]:
class ToyDataset(Dataset):
    def __init__(self, data):
        super(ToyDataset, self).__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        sample = torch.tensor(self.data[item], dtype=torch.float32, device=device)
        return sample, -1.

In [8]:
class ToyEncoder(nn.Module):
    def __init__(self, d):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(1, 2*d),
            nn.LeakyReLU(),
            nn.Linear(2*d, 2*d),
        )

    def forward(self, x):
        return self.net(x)

    
class ToyDecoder(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.aux = nn.Parameter(torch.tensor(0., dtype=torch.float32))
        self.log_alpha = nn.Parameter(torch.tensor(0, device=device, dtype=torch.float32))
        self.log_beta = nn.Parameter(torch.tensor(0., device=device, dtype=torch.float32))
        self.sigma = sigma

    @property
    def alpha(self,):
        return torch.exp(self.log_alpha)
    
    @property
    def beta(self,):
        return torch.exp(self.log_beta)

    def forward(self, x):
        return self.alpha * (torch.sqrt(torch.sum(torch.pow(x, 2), dim=1, keepdim=True)) + self.beta) # + torch.randn_like(x[:, :1]) * self.sigma + 0. * self.aux
    
class TrueDecoder(nn.Module):
    def __init__(self, d, sigma, eps=None):
        super().__init__()
        self.register_buffer('eps', torch.tensor(eps, dtype=torch.float32))
        self.aux = nn.Parameter(torch.tensor(0., dtype=torch.float32))
        self.sigma = sigma

    def forward(self, z):
        return 2 * np.pi * (torch.sqrt(torch.sum(torch.pow(z, 2), dim=1, keepdim=True)) + 2.) # + torch.randn_like(z[:, :1]) * self.sigma + 0. * self.aux

In [9]:
N = 10000
d = 2
sigma = 1.
eps = 2 + np.random.randn(1, d)
X_train = generate_dataset(N=N, eps0=eps, d=d, sigma=sigma)
X_val = generate_dataset(N=N // 100, eps0=eps, d=d, sigma=sigma)

In [10]:
train_dataset = ToyDataset(data=X_train)
val_dataset = ToyDataset(data=X_val)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True,)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)

In [12]:
def replace_enc_dec(model):
    model.encoder_net = ToyEncoder(d=d)
    model.decoder_net = TrueDecoder(d=d, eps=eps, sigma=sigma)
    model = model.to(device)
    return model

In [13]:
#----- VAE ------ #
seed_everything(42)
vae = VAE_Toy(shape=28, act_func=nn.LeakyReLU,
            num_samples=1, hidden_dim=d,
            net_type='conv', dataset='toy')
vae = replace_enc_dec(vae)


# ----- IWAE ------ #
seed_everything(42)
iwae = IWAE_Toy(shape=28, act_func=nn.LeakyReLU,
            num_samples=50, hidden_dim=d,
            net_type='conv', dataset='toy')
iwae = replace_enc_dec(iwae)
iwae.name = 'IWAE'

# ----- LMCVAE ----- #
seed_everything(42)
lmcvae = LMCVAE_Toy(shape=28, act_func=nn.LeakyReLU,
            num_samples=1, hidden_dim=d,
            net_type='conv', dataset='toy',
            step_size=0.01, K=5, use_transforms=False, learnable_transitions=False, return_pre_alphas=True, use_score_matching=False,
                      ula_skip_threshold=0.1, grad_skip_val=0., grad_clip_val=0., use_cloned_decoder=False, variance_sensitive_step=True,
                     acceptance_rate_target=0.9, annealing_scheme='all_learnable')
lmcvae = replace_enc_dec(lmcvae)
lmcvae.name = 'LMCVAE'

# ----- AMCVAE ----- #
seed_everything(42)
amcvae = AMCVAE_Toy(shape=28, act_func=nn.LeakyReLU,
            num_samples=1, hidden_dim=d,
            net_type='conv', dataset='toy',
            step_size=0.01, K=5, use_barker=False, learnable_transitions=False, use_alpha_annealing=True, grad_skip_val=0.,
                      grad_clip_val=0., use_cloned_decoder=False, variance_sensitive_step=True,
                     acceptance_rate_target=0.8, annealing_scheme='all_learnable')
amcvae = replace_enc_dec(amcvae)
amcvae.name = 'AMCVAE'


# ----- VAE_with_Flows ----- #
seed_everything(42)
flows_vae = VAE_with_flows_Toy(shape=28, act_func=nn.LeakyReLU,
            num_samples=1, hidden_dim=d,
            net_type='conv', dataset='toy',
            flow_type='RNVP', num_flows=5, need_permute=True)
flows_vae = replace_enc_dec(flows_vae)
flows_vae.name = 'VAE_with_Flows'

In [14]:
def run_trainer(model):
    seed_everything(42)
    tb_logger = pl_loggers.TensorBoardLogger('lightning_logs/')
    trainer = pl.Trainer(logger=tb_logger, fast_dev_run=False, max_epochs=31, automatic_optimization=True, )
    trainer.fit(model, train_dataloader=train_loader, val_dataloaders=val_loader)

In [15]:
run_trainer(flows_vae)

GPU available: True, used: False
TPU available: None, using: 0 TPU cores
/home/nkotelevskii/anaconda3/envs/condatorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)
/home/nkotelevskii/anaconda3/envs/condatorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Disable automatic optimization with the trainer flag is deprecated and will be removed in v1.3.0!Please use the property on the LightningModule for disabling automatic optimization
  warnings.warn(*args, **kwargs)

  | Name            | Type        | Params
------------------------------------------------
0 | encoder_net     | ToyEncoder  | 28    
1 | decoder_net     | TrueDecoder | 1     
2 | transitions_nll | ModuleList  | 8     
3 | Flow            | NormFlow    | 50    
------------------------------------------------
79        Trainable params


/home/nkotelevskii/anaconda3/envs/condatorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


In [16]:
run_trainer(amcvae)

GPU available: True, used: False
TPU available: None, using: 0 TPU cores

  | Name            | Type        | Params
------------------------------------------------
0 | encoder_net     | ToyEncoder  | 28    
1 | decoder_net     | TrueDecoder | 1     
2 | transitions_nll | ModuleList  | 8     
3 | transitions     | ModuleList  | 5     
------------------------------------------------
34        Trainable params
13        Non-trainable params
47        Total params


In [17]:
run_trainer(lmcvae)

GPU available: True, used: False
TPU available: None, using: 0 TPU cores

  | Name            | Type        | Params
------------------------------------------------
0 | encoder_net     | ToyEncoder  | 28    
1 | decoder_net     | TrueDecoder | 1     
2 | transitions_nll | ModuleList  | 8     
3 | transitions     | ModuleList  | 5     
------------------------------------------------
34        Trainable params
13        Non-trainable params
47        Total params


In [18]:
run_trainer(iwae)

GPU available: True, used: False
TPU available: None, using: 0 TPU cores

  | Name            | Type        | Params
------------------------------------------------
0 | encoder_net     | ToyEncoder  | 28    
1 | decoder_net     | TrueDecoder | 1     
2 | transitions_nll | ModuleList  | 8     
------------------------------------------------
29        Trainable params
8         Non-trainable params
37        Total params


In [19]:
run_trainer(vae)

GPU available: True, used: False
TPU available: None, using: 0 TPU cores

  | Name            | Type        | Params
------------------------------------------------
0 | encoder_net     | ToyEncoder  | 28    
1 | decoder_net     | TrueDecoder | 1     
2 | transitions_nll | ModuleList  | 8     
------------------------------------------------
29        Trainable params
8         Non-trainable params
37        Total params


In [20]:
n_leapfrogs = 5
step_size = 0.01
n_samples = 10000
hmc = HMC(n_leapfrogs=n_leapfrogs, step_size=step_size, partial_ref=False, use_barker=False).to(device)

In [21]:
X_item = torch.tensor([[2 * np.pi * (2. + 2.)]], device=device, dtype=torch.float32)

In [22]:
## Receive true posterior sample:
target_density = lambda z, x: torch.distributions.Normal(loc=2 * np.pi * (torch.sqrt(torch.sum(torch.pow(z, 2), dim=1, keepdim=True)) + 2.), scale=sigma).log_prob(X_item.repeat(n_samples, 1)).sum(
    -1) + torch.distributions.Normal(loc=torch.tensor(0., device=device, dtype=torch.float32),
                                     scale=torch.tensor(1., device=device, dtype=torch.float32)).log_prob(z).sum(-1)

true_posterior_samples = run_chain(kernel=hmc, z_init=torch.randn(n_samples, 2, device=device), target=target_density, return_trace=False, n_steps=1, burnin=500).cpu().numpy()

In [23]:
def receive_model_samples(model, X_item, stacked=False):
    with torch.no_grad():
        x = X_item
        model_samples, mu, logvar = model.enc_rep(x=x, n_samples=n_samples)
        if model.name in ['LMCVAE', 'AMCVAE']:
            model_samples_init = model_samples
            model_samples = model.run_transitions(z=model_samples, x=x.repeat(n_samples, 1), mu=mu, logvar=logvar)[0]
        elif model.name in ['VAE_with_Flows']:
            model_samples = model.Flow(model_samples)[0]
    return model_samples.cpu().numpy()

def plot_contours(model, X_item, stacked=False, graph=None):
    x = X_item
    with torch.no_grad():
        model_samples, mu, logvar = model.enc_rep(x=x, n_samples=n_samples)
        logprobs = torch.distributions.Normal(loc=mu, scale=torch.exp(0.5 * logvar)).log_prob(model_samples).sum(-1)
#     plt.contour(model_samples[:, 0].cpu()[..., None], model_samples[:, 1].cpu()[..., None], logprobs.cpu()[..., None])
    if graph is None:
        plt.scatter(model_samples[:, 0].cpu(), model_samples[:, 1].cpu(),)
    else:
        graph.x = model_samples[:, 0].cpu()
        graph.y = model_samples[:, 1].cpu()
        graph.plot_joint(plt.scatter, marker='x', c='g', s=50, alpha=0.5)
    

def receive_posterior_samples(model, n_samples, stacked=False):
    with torch.no_grad():
        model_target_density = lambda z, x: model.joint_logdensity()(z=z, x=X_item.repeat(n_samples, 1))
        model_posterior_samples = run_chain(kernel=hmc, z_init=torch.randn(n_samples, 2, device=device), target=model_target_density, return_trace=False, n_steps=1, burnin=500).cpu().numpy()
    return model_posterior_samples

In [24]:
vae_sample = receive_model_samples(vae, X_item)
vae_posterior_sample = receive_posterior_samples(vae, n_samples)

iwae_sample = receive_model_samples(iwae, X_item)
iwae_posterior_sample = receive_posterior_samples(iwae, n_samples)

lmcvae_sample = receive_model_samples(lmcvae, X_item)
lmcvae_posterior_sample = receive_posterior_samples(lmcvae, n_samples)

amcvae_sample = receive_model_samples(amcvae, X_item)
amcvae_posterior_sample = receive_posterior_samples(amcvae, n_samples)

flows_vae_sample = receive_model_samples(flows_vae, X_item)
flows_vae_posterior_sample = receive_posterior_samples(flows_vae, n_samples)

In [30]:
# plot
# ========================================
graph = sns.jointplot(x=true_posterior_samples[:, 0], y=true_posterior_samples[:, 1], color='r', alpha=0.5);
# graph = sns.jointplot(x=vae_posterior_sample[:, 0], y=vae_posterior_sample[:, 1], color='r', alpha=0.5);
# graph = sns.jointplot(x=iwae_posterior_sample[:, 0], y=iwae_posterior_sample[:, 1], color='r', alpha=0.5);
# graph = sns.jointplot(x=lmcvae_posterior_sample[:, 0], y=lmcvae_posterior_sample[:, 1], color='r', alpha=0.5);
# graph = sns.jointplot(x=amcvae_posterior_sample[:, 0], y=amcvae_posterior_sample[:, 1], color='r', alpha=0.5);
# graph = sns.jointplot(x=flows_vae_posterior_sample[:, 0], y=flows_vae_posterior_sample[:, 1], color='r', alpha=0.5);


# graph.x = vae_sample[:, 0]
# graph.y = vae_sample[:, 1]
# graph.plot_joint(plt.scatter, marker='x', c='g', s=50, alpha=0.5);


# graph.x = iwae_sample[:, 0]
# graph.y = iwae_sample[:, 1]
# graph.plot_joint(plt.scatter, marker='x', c='b', s=50, alpha=0.5);


# graph.x = lmcvae_sample[:, 0]
# graph.y = lmcvae_sample[:, 1]
# graph.plot_joint(plt.scatter, marker='x', c='y', s=50, alpha=0.1);


graph.x = amcvae_sample[:, 0]
graph.y = amcvae_sample[:, 1]
graph.plot_joint(plt.scatter, marker='x', c='purple', s=50, alpha=0.5);


# graph.x = flows_vae_sample[:, 0]
# graph.y = flows_vae_sample[:, 1]
# graph.plot_joint(plt.scatter, marker='x', c='orange', s=50, alpha=0.5);



# plt.xlim(-5., 5.)
# plt.ylim(-5., 5.)
# plt.axis('equal');


# plot_contours(model=stacked_vae, X_item=X_item, stacked=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
def plot_heatmap(samples, x_limits, y_limits, title=None, gamma=0.75, name='default.png'):
    import matplotlib.colors as mcolors
    from scipy.stats import kde
    plt.close()
    plt.figure(figsize=(5, 5), dpi=300)
    
    x = samples[:, 0]
    y = samples[:, 1]
    nbins = 300
    k = kde.gaussian_kde([x,y], bw_method=0.1)
    xi, yi = np.mgrid[x_limits[0]:x_limits[1]:nbins*1j, y_limits[0]:y_limits[1]:nbins*1j]
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))

    # Make the plot
    plt.pcolormesh(xi, yi, zi.reshape(xi.shape))
    plt.axis('off')
    plt.xlim((x_limits[0], x_limits[1]))
    plt.ylim((y_limits[0], y_limits[1]))
#     plt.axis('equal')
    plt.tight_layout()
    plt.savefig(name, format='png')
    plt.show()

In [27]:
plot_heatmap(true_posterior_samples, (-4, 4), (-4, 4), title=None, gamma=0.5, name='true_posterior.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-26-d92f338a268b>:16: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(xi, yi, zi.reshape(xi.shape))


In [28]:
plot_heatmap(amcvae_sample, (-4, 4), (-4, 4), title=None, gamma=0.5, name='amcvae.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-26-d92f338a268b>:16: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(xi, yi, zi.reshape(xi.shape))
